In [1]:
import cv2
import numpy as np
import random

seed=123
np.random.seed(seed)
random.seed(seed)

from imageio import imread, imsave

from os.path import isdir, join, isfile
from os import listdir
from utils.images_utils import mostrar, read_images
from utils.image_modification import batch_RGB2LAB,batch_RGB2LAB,batch_RGB2Opponent,batch_RGB2GRAY,calcular_modulo_gradiente

In [2]:
def read_train_data():
    template_src_raw = 'dataset/Segmentation_Rigid_Training/Training/OP{}/Raw/img_{:02}_raw.png'
    template_src_mask = 'dataset/Segmentation_Rigid_Training/Training/OP{}/Masks/img_{:02}_instrument.png'
    
    raws = []
    masks = []
    for op in range(1,5):
        for img in range(1,41):
            
            raws.append(imread(template_src_raw.format(op,img)))
            masks.append(imread(template_src_mask.format(op,img), as_gray=True))
            
    return np.array(raws), np.array(masks)
    
raws, masks = read_train_data()

In [3]:
masks[masks!=0]=1

In [4]:
n,hh,ww,c = raws.shape
LAB = batch_RGB2LAB(raws)
HSV = batch_RGB2LAB(raws)
opponent = batch_RGB2Opponent(raws)
gray = batch_RGB2GRAY(raws)
G = calcular_modulo_gradiente(gray).reshape((n,hh,ww,1))
gray = gray.reshape((n,hh,ww,1))
HSV[:,:,:,1]*=255

In [5]:
print(raws.shape)
print(LAB.shape)
print(HSV.shape)
print(opponent.shape)
print(gray.shape)
print(G.shape)

(160, 480, 640, 3)
(160, 480, 640, 3)
(160, 480, 640, 3)
(160, 480, 640, 3)
(160, 480, 640, 1)
(160, 480, 640, 1)


In [6]:
X = np.concatenate((raws, gray, G, HSV,LAB, opponent), axis=3)
print(X.shape)

(160, 480, 640, 14)


In [7]:
del(LAB)
del(HSV)
del(opponent)
del(gray)
del(G)

xx =  X.reshape((n*hh*ww, X.shape[-1]))
yy =  masks.reshape((n*hh*ww,))
del(X)
del(masks)
del(raws)

In [8]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from pandas import DataFrame as df

In [9]:
print(xx.shape)
print(yy.shape)
X_train, X_test, y_train, y_test =  train_test_split(xx, yy, test_size=0.8, random_state=123, stratify=yy)
del(xx)
del(yy)
del(X_test)
del(y_test)

(49152000, 14)
(49152000,)


In [10]:
columns = ['Red', 'Green','Blue','gray','Gmod','H','S','V','L','A', 'B', 'Op0', 'Op1','Op2', 'label']
data_train = df(data=np.hstack((X_train, y_train.reshape((y_train.shape[0],1)))), columns=columns);

In [ ]:
from pandas.plotting import scatter_matrix
del(columns)
del(X_train)
del(y_train)

colors=['red','green']

# make a scatter plot
#scatter_matrix(df,figsize=[20,20],marker='x',c=df.Survived.apply(lambda x:colors[x]))


scatter_matrix(data_train, alpha=0.2, figsize=(30, 20), diagonal='kde', c=data_train.label.apply(lambda x:colors[int(x)]));

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(data_train)